# Simple Model


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_pickle('/content/drive/MyDrive/t10df_20210719.pkl')

#dfall = pd.read_csv('/content/drive/MyDrive/dfdist66037minlns4573.csv',index_col=0)
#grouped=dfall.groupby(dfall['clusterId'])
#df_0 = grouped.get_group(0).copy()
#df_1 = grouped.get_group(1).copy()
#df_2 = grouped.get_group(2).copy()
#tags_of_0=df_0['tags'].unique()
#tags_of_1=df_1['tags'].unique()
#tags_of_2=df_2['tags'].unique()
data

import warnings
warnings.filterwarnings('ignore')

import glob
import shutil
import pickle
import os
import time
from pandas import concat
import matplotlib.pyplot as plt
import random
#pip install -U scikit-learn scipy matplotlib
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import math





data['column_mmsi'] = np.where(data["mmsi(t-1)"] == data["mmsi(t)"], True, False)
false_count = (~data.column_mmsi).sum()
true_count = (data.column_mmsi).sum()
print("num of false mmsi",(false_count))
print("num of true mmsi",(true_count))

data['column_id'] = np.where(data["id(t-1)"] == data["id(t)"], True, False)
false_count_id = (~data.column_id).sum()
true_count_id = (data.column_id).sum()
print("num of false id",(false_count_id))
print("num of true id",(true_count_id))

#to id dinei to trip (mporw na xrhsimopoihsw ws timegap 1 h)

data = data[data['column_mmsi'] == True] #drop rows where column id is false, id(t)=/=id(t-1)
data['column_mmsi'] = np.where(data["mmsi(t-1)"] == data["mmsi(t)"], True, False)
false_count = (~data.column_mmsi).sum()
true_count = (data.column_mmsi).sum()
print("num of false mmsi",(false_count))
print("num of true mmsi",(true_count))

data['column_id'] = np.where(data["id(t-1)"] == data["id(t)"], True, False)
false_count_id = (~data.column_id).sum()
true_count_id = (data.column_id).sum()
print("num of false id",(false_count_id))
print("num of true id",(true_count_id))
print("now mmsi(t) == mmsi(t-1) ")
#shift me group, short me xrono

df0=data.copy()
df0 = df0[df0.columns.drop(list(df0.filter(regex='mmsi')))]
df0 = df0[df0.columns.drop(list(df0.filter(regex='dataset_tr1_val2_test3_augm_0_norm')))]
df0 = df0[df0.columns.drop(list(df0.filter(regex='id')))]
df0 = df0[df0.columns.drop(list(df0.filter(regex='WGS84')))]
df0 = df0[df0.columns.drop(list(df0.filter(regex='dist_m')))]


df0=df0.drop(labels=['t(t-10)', 't(t-9)','t(t-8)','t(t-7)',
                 't(t-6)','t(t-5)','t(t-4)', 't(t-3)','t(t-2)','t(t-1)','t(t)','dt(t-10)'], axis=1) 

df0=df0.drop(labels=['dataset_tr1_val2_test3_augm_0(t-9)', 'dataset_tr1_val2_test3_augm_0(t-8)',
                     'dataset_tr1_val2_test3_augm_0(t-7)','dataset_tr1_val2_test3_augm_0(t-6)',
                 'dataset_tr1_val2_test3_augm_0(t-5)','dataset_tr1_val2_test3_augm_0(t-4)',
                     'dataset_tr1_val2_test3_augm_0(t-3)', 'dataset_tr1_val2_test3_augm_0(t-2)',
                     'dataset_tr1_val2_test3_augm_0(t-1)','dataset_tr1_val2_test3_augm_0(t)'], axis=1) 


grouped=df0.groupby(df0['dataset_tr1_val2_test3_augm_0(t-10)'])
#training data shenanigans
df_tr = grouped.get_group(1.0)
df_tr1=df_tr.copy()
#to thelw gia to telos
dftrainzlon=df_tr1[["lon(t-1)"]]
dftrainzlat=df_tr1[["lat(t-1)"]]
#drop lon lats
df_trx=df_tr.filter(regex=("^dlon|^dlat*|^speed|^dt"))
df_trx=df_trx.drop(labels=['dlon(t)', 'dlat(t)'], axis=1)
df_try=df_tr1[["dlon(t)","dlat(t)"]]

#validation data shenanigans
df_val= grouped.get_group(2.0)
df_val1=df_val.copy()
#to thelw just in case
dfvalzlon=df_val1[["lon(t-1)"]]
dfvalzlat=df_val1[["lat(t-1)"]]
#drop lon lats
df_valx=df_val.filter(regex=("^dlon|^dlat*|^speed|^dt"))
df_valx=df_valx.drop(labels=['dlon(t)', 'dlat(t)'], axis=1)
df_valy=df_val1[["dlon(t)","dlat(t)"]]

#testing data shenanigans
df_test = grouped.get_group(3.0)
df_test1=df_test.copy()
#to thelw gia to telos
#edw eixa kanei to lathos na balw dftestzlon=df_VAL1[["lon(t-1)"]] 
dftestzlon=df_test1[["lon(t-1)"]]
dftestzlat=df_test1[["lat(t-1)"]]
#drop lon lats
df_testx=df_test.filter(regex=("^dlon|^dlat*|^speed|^dt"))
df_testx=df_testx.drop(labels=['dlon(t)', 'dlat(t)'], axis=1)
df_testy =df_test1[["dlon(t)","dlat(t)"]]



from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler1 = StandardScaler()#StandardScaler(),MinMaxScaler(feature_range=(0, 1))
scaler2 = StandardScaler()
scaler3 = StandardScaler()
scaler4 = StandardScaler()
scaler5 = StandardScaler()
scaler6 = StandardScaler()


columns=['dlon(t-10)','dlat(t-10)','speed(t-10)','dlon(t-9)','dlat(t-9)','dt(t-9)','speed(t-9)','dlon(t-8)','dlat(t-8)','dt(t-8)',
         'speed(t-8)','dlon(t-7)','dlat(t-7)','dt(t-7)','speed(t-7)','dlon(t-6)','dlat(t-6)','dt(t-6)','speed(t-6)','dlon(t-5)','dlat(t-5)',
         'dt(t-5)','speed(t-5)','dlon(t-4)','dlat(t-4)','dt(t-4)','speed(t-4)','dlon(t-3)','dlat(t-3)','dt(t-3)','speed(t-3)','dlon(t-2)',
         'dlat(t-2)','dt(t-2)','speed(t-2)','dlon(t-1)','dlat(t-1)','dt(t-1)','speed(t-1)','dt(t)','speed(t)']

df_trx[columns]=scaler1.fit_transform(df_trx[columns])
df_try[['dlon(t)','dlat(t)']]=scaler2.fit_transform(df_try[['dlon(t)','dlat(t)']])
df_valx[columns]=scaler3.fit_transform(df_valx[columns])
df_valy[['dlon(t)','dlat(t)']]=scaler4.fit_transform(df_valy[['dlon(t)','dlat(t)']])
df_testx[columns]=scaler5.fit_transform(df_testx[columns])
df_testy[['dlon(t)','dlat(t)']]=scaler6.fit_transform(df_testy[['dlon(t)','dlat(t)']])







 

num of false mmsi 0
num of true mmsi 57616
num of false id 0
num of true id 57616
num of false mmsi 0
num of true mmsi 57616
num of false id 0
num of true id 57616
now mmsi(t) == mmsi(t-1) 


In [ ]:
from sklearn.metrics import mean_absolute_error
layer1='relu'
    # Building a model with SimpleRNN 
from keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.layers import (BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense)
#lets try Dropout BatchNormalization shuflle  relu elu esu 
#define model and model parameters
batch_size = 32
model = Sequential()
n_cols=df_trx.shape[1]
n_rows=df_trx.shape[0]
epochs=1000
patience=10

model.add(Dense(units=32, input_shape=(n_cols, ), activation=layer1))#relu,selu,sigmoid,tanh
model.add(Dense(16, activation=layer1))
model.add(Dense(8, activation=layer1))
model.add(Dense(4, activation=layer1))
model.add(Dense(2, activation=layer1))

#compile model
model.compile(loss='mae', optimizer='adam',metrics=['mean_absolute_error']) 

#fit model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, 
                               save_weights_only=True, monitor='val_mean_absolute_error', mode='min', verbose=1)

early_stopping_monitor = EarlyStopping(patience=patience)
history=model.fit(df_trx, df_try, validation_data=(df_valx,df_valy), epochs=epochs, 
                  callbacks=[early_stopping_monitor,checkpointer], batch_size=batch_size, verbose=1,shuffle=True)
model.summary()
# list all data in history
print(history.history.keys())
#metrics to use accuracy, means_squared_error



Epoch 1/1000
804/811 [============================>.] - ETA: 0s - loss: 0.5175 - mean_absolute_error: 0.5175
Epoch 1: val_mean_absolute_error improved from inf to 0.55072, saving model to weights.hdf5
811/811 [==============================] - 5s 4ms/step - loss: 0.5173 - mean_absolute_error: 0.5173 - val_loss: 0.5507 - val_mean_absolute_error: 0.5507
Epoch 2/1000
790/811 [============================>.] - ETA: 0s - loss: 0.4643 - mean_absolute_error: 0.4643
Epoch 2: val_mean_absolute_error improved from 0.55072 to 0.52530, saving model to weights.hdf5
811/811 [==============================] - 3s 3ms/step - loss: 0.4637 - mean_absolute_error: 0.4637 - val_loss: 0.5253 - val_mean_absolute_error: 0.5253
Epoch 3/1000
810/811 [============================>.] - ETA: 0s - loss: 0.4516 - mean_absolute_error: 0.4516
Epoch 3: val_mean_absolute_error improved from 0.52530 to 0.51883, saving model to weights.hdf5
811/811 [==============================] - 2s 3ms/step - loss: 0.4516 - mean_absolu

In [ ]:
from sklearn.metrics import mean_absolute_error


# Predicting and plotting the result
trainPredict = model.predict(df_trx, batch_size=batch_size)
testPredict = model.predict(df_testx, batch_size=batch_size)

# invert predictions
#(25925, 1) (21585, 1)
trainPredict = np.reshape(trainPredict, (-1, 2))
ytr2d = np.reshape(df_try, (-1, 2))

testPredict = np.reshape(testPredict, (-1, 2))
yte2d = np.reshape(df_testy, (-1, 2))
trainPredict


trainPredict = scaler2.inverse_transform(trainPredict)
trainY = scaler2.inverse_transform(ytr2d)

testPredict = scaler6.inverse_transform(testPredict)
testY = scaler6.inverse_transform(yte2d)

dfpredict = pd.DataFrame(testPredict,columns=['dlon','dlat'])
dftest= pd.DataFrame(testY,columns=['dlon','dlat'])
trainPred= pd.DataFrame(trainPredict,columns=['dlon','dlat'])
trainReal= pd.DataFrame(trainY,columns=['dlon','dlat'])



dfpredictdlon= pd.DataFrame([])
dftestdlon= pd.DataFrame([])
trainPreddlon =pd.DataFrame([])
trainRealdlon = pd.DataFrame([])
dfpredictdlat= pd.DataFrame([])
dftestdlat= pd.DataFrame([])
trainPreddlat = pd.DataFrame([])
trainRealdlat =pd.DataFrame([])

dfpredictdlon['dlon']=dfpredict['dlon']
dftestdlon['dlon']= dftest['dlon']
trainPreddlon['dlon'] = trainPred['dlon']
trainRealdlon['dlon'] = trainReal['dlon']
dfpredictdlat['dlat']= dfpredict['dlat']
dftestdlat['dlat']= dftest['dlat']
trainPreddlat['dlat'] = trainPred['dlat']
trainRealdlat['dlat'] = trainReal['dlat']


dfpredictdlon['lon']=dftestzlon["lon(t-1)"].values
dfpredictdlat['lat'] =dftestzlat["lat(t-1)"].values
dftestdlon['lon']=dftestzlon["lon(t-1)"].values
dftestdlat['lat'] =dftestzlat["lat(t-1)"].values

trainPreddlon['lon'] =dftrainzlon["lon(t-1)"].values
trainRealdlon['lon'] =dftrainzlon["lon(t-1)"].values
trainPreddlat['lat'] =dftrainzlat["lat(t-1)"].values
trainRealdlat['lat'] =dftrainzlat["lat(t-1)"].values



#df.sum(axis = 1, skipna = True)

trainPredlon= pd.DataFrame([])
trainPredlat= pd.DataFrame([])
trainReallon= pd.DataFrame([])
trainReallat= pd.DataFrame([])
dfpredictlon= pd.DataFrame([])
dfpredictlat= pd.DataFrame([])
dftestlon= pd.DataFrame([])
dftestlat= pd.DataFrame([])
dfpredictdlon['flon']=dfpredictdlon.sum(axis = 1, skipna = True)
dfpredictdlat['flat']= dfpredictdlat.sum(axis = 1, skipna = True)
dftestdlon['flon']= dftestdlon.sum(axis = 1, skipna = True)
dftestdlat['flat']= dftestdlat.sum(axis = 1, skipna = True)

trainPreddlon['flon']= trainPreddlon.sum(axis = 1, skipna = True)
trainPreddlat['flat']= trainPreddlat.sum(axis = 1, skipna = True)
trainRealdlon['flon']= trainRealdlon.sum(axis = 1, skipna = True)
trainRealdlat['flat']= trainRealdlat.sum(axis = 1, skipna = True)

trainPredlon['lon']= trainPreddlon['flon']
trainPredlat['lat']= trainPreddlat['flat']
trainReallon['lon']= trainRealdlon['flon']
trainReallat['lat']= trainRealdlat['flat']
dfpredictlon['lon']=dfpredictdlon['flon']
dfpredictlat['lat']=dfpredictdlat['flat']
dftestlon['lon']=dftestdlon['flon']
dftestlat['lat']=dftestdlat['flat']

print("Test northing MSE: ", mean_squared_error(dftestlat, dfpredictlat))

print("Test northing R2: ", r2_score(dftestlon, dfpredictlon))

print("Test easting MSE: ", mean_squared_error(dftestlon, dfpredictlon))

print("Test easting R2: ", r2_score(dftestlat, dfpredictlat))

print("Test northing MAE: ", mean_absolute_error(dftestlat, dfpredictlat))

print("Test easting MAE: ", mean_absolute_error(dftestlon, dfpredictlon))

df_predtrainresults= pd.DataFrame([])
df_trainresults = pd.DataFrame([])
df1= pd.DataFrame([])

df_predtrainresults["long_latPREDICT"] = list(zip(trainPredlon['lon'], trainPredlat['lat']))
df_trainresults["long_latRESULT"] = list(zip(trainReallon['lon'], trainReallat['lat']))

df1['prediction']=df_predtrainresults["long_latPREDICT"]
df1['reality']=df_trainresults["long_latRESULT"]

def distance(df1):
  return math.dist(df1['prediction'],df1['reality'])

df1["distance(m)"] = df1.apply(distance, axis =1)
df1["distance(m)"].mean()
#18886.096412101044 meso sfalma sto train set 18,88609 km  me ola ta matzounia kai 5 epochs

df_predresults = pd.DataFrame([])
df_testresults = pd.DataFrame([])
df= pd.DataFrame([])
df_predresults["long_latPREDICT"] = list(zip(dfpredictlon['lon'], dfpredictlat['lat']))
df_testresults["long_latRESULT"] = list(zip(dftestlon['lon'], dftestlat['lat']))
#!pip install haversine
#from haversine import haversine, Unit

#arr_pred = df_predresults.to_numpy()
#arr_test = df_testresults.to_numpy()

df['prediction']=df_predresults["long_latPREDICT"]
df['reality']=df_testresults["long_latRESULT"]


def distance(df):
  return math.dist(df['prediction'],df['reality'])

df["distance(m)"] = df.apply(distance, axis =1)
df["distance(m)"].mean()


testxdf = scaler5.inverse_transform(df_testx)


dfftestx = pd.DataFrame(testxdf,columns=columns)

df["dt"]=dfftestx['dt(t)']
df
max(df['distance(m)'])


index1=0
index2=0
index3=0
index4=0
index5=0
index6=0
index7=0
index8=0
index9=0
index10=0
index11=0
index12=0
index13=0
index14=0
index15=0
index16=0
index17=0
index18=0
index19=0
index20=0
index21=0
index22=0
index23=0
index24=0
index25=0
index26=0
index27=0
index28=0
index29=0
dist1740=0
dist1680=0
dist1620=0
dist1560=0
dist1500=0
dist1440=0
dist1380=0
dist1320=0
dist1260=0
dist1200=0
dist1140=0
dist1080=0
dist1020=0
dist960=0
dist900=0
dist840=0
dist780=0
dist720=0
dist660=0
dist600=0
dist540=0
dist480=0
dist420=0
dist360=0
dist300=0
dist240=0
dist180=0
dist120=0
dist60=0
for index, row in df.iterrows(): 
  window= row[3]
  if window>=1740 and window<=1800:
          dist1740=dist1740+row[2] 
          index1=index1+1
  if window>=1680 and window<1740:
          dist1680=dist1680+row[2] 
          index2=index2+1
  if window>=1620 and window<1680:
          dist1620=dist1620+row[2] 
          index3=index3+1
  if window>=1560 and window<1620:
          dist1560=dist1560+row[2] 
          index4=index4+1
  if window>=1500 and window<1560:
          dist1500=dist1500+row[2] 
          index5=index5+1
  if window>=1440 and window<1500:
          dist1440=dist1440+row[2] 
          index6=index6+1
  if window>=1380 and window<1440:
          dist1380=dist1380+row[2] 
          index7=index7+1
  if window>=1320 and window<1380:
          dist1320=dist1320+row[2] 
          index8=index8+1
  if window>=1260 and window<1320:
          dist1260=dist1260+row[2] 
          index9=index9+1    
  if window>=1200 and window<1260:
          dist1200=dist1200+row[2] 
          index10=index10+1
  if window>=1140 and window<1200:
          dist1140=dist1140+row[2] 
          index11=index11+1
  if window>=1080 and window<1140:
          dist1080=dist1080+row[2] 
          index12=index12+1
  if window>=1020 and window<1080:
          dist1020=dist1020+row[2] 
          index13=index13+1
  if window>=960 and window<1020:
          dist960=dist960+row[2] 
          index14=index14+1
  if window>=900 and window<960:
          dist900=dist900+row[2] 
          index15=index15+1
  if window>=840 and window<900:
          dist840=dist840+row[2] 
          index16=index16+1
  if window>=780 and window<840:
          dist780=dist780+row[2] 
          index17=index17+1
  if window>=720 and window<780:
          dist720=dist720+row[2] 
          index18=index18+1
  if window>=660 and window<720:
          dist660=dist660+row[2] 
          index19=index19+1
  if window>=600 and window<660:
          dist600=dist600+row[2] 
          index20=index20+1
  if window>=540 and window<600:
          dist540=dist540+row[2] 
          index21=index21+1
  if window>=480 and window<540:
          dist480=dist480+row[2] 
          index22=index22+1
  if window>=420 and window<480:
          dist420=dist420+row[2] 
          index23=index23+1
  if window>=360 and window<420:
          dist360=dist360+row[2] 
          index24=index24+1
  if window>=300 and window<360:
          dist300=dist300+row[2] 
          index25=index25+1
  if window>=240 and window<300:
          dist240=dist240+row[2] 
          index26=index26+1
  if window>=180 and window<240:
          dist180=dist180+row[2] 
          index27=index27+1
  if window>=120 and window<180:
          dist120=dist120+row[2] 
          index28=index28+1
  if window>=0 and window<120:
          dist60=dist60+row[2] 
          index29=index29+1
        
#dist1740,dist1680,dist1620,dist1560,dist1500,dist1440,dist1380,dist1320,dist1200,dist1200,dist1140,
#dist1080,dist1020,dist960,dist900,dist840,dist780,dist720,dist660,dist600,dist540,dist480,dist420,
#dist360,dist300,dist240,dist180,dist120,dist60

print( '[0,2)&' ,(dist60/index29)/1000, index29)
print( '[2,3)&' ,(dist120/index28)/1000,index28)
print( '[3,4)&' ,(dist180/index27)/1000,index27)  
print( '[4,5)&' ,(dist240/index26)/1000,index26)  
print( '[5,6)&' ,(dist300/index25) /1000,index25) 
print( '[6,7)&' ,(dist360/index24) /1000,index24) 
print( '[7,8)&' ,(dist420/index23)/1000,index23)
print( '[8,9)&' ,(dist480/index22) /1000,index22) 
print( '[9,10)&' ,(dist540/index21) /1000,index21) 
print( '[10,11)&' ,(dist600/index20)/1000,index20) 
print( '[11,12)&' ,(dist660/index19) /1000,index19) 
print( '[12,13)&' ,(dist720/index18)/1000,index18)
print( '[13,14)&' ,(dist780 /index17) /1000,index17) 
print( '[14,15)&' ,(dist840/index16) /1000,index16) 
print( '[15,16)&' ,(dist900/index15)/1000,index15) 
print( '[16,17)&' ,(dist960/index14) /1000,index14) 
print( '[17,18)&' ,(dist1020/index13)/1000,index13)
print( '[18,19)&' ,(dist1080/index12) /1000,index12) 
print( '[19,20)&' ,(dist1140/index11) /1000,index11) 
print( '[20,21)&' ,(dist1200/index10)/1000,index10) 
print( '[21,22)&' ,(dist1260/index9)  /1000,index9)
print( '[22,23)&' ,(dist1320 /index8)/1000,index8)
print( '[23,24)&' ,(dist1380/index7) /1000,index7) 
print( '[24,25)&' ,(dist1440/index6)/1000,index6)  
print( '[25,26)&' ,(dist1500/index5)/1000,index5) 
print( '[26,27)&' ,(dist1560/index4)/1000,index4)  
print( '[27,28)&' ,(dist1620/index3)/1000,index3)
print( '[28,29)&' ,(dist1680/index2)/1000,index2)  
print( '[29,30)&' ,(dist1740/index1)/1000,index1)  

675/675 [==============================] - 1s 1ms/step
Test northing MSE:  468899.18983213103
Test northing R2:  0.9996416340163309
Test easting MSE:  855112.1083481195
Test easting R2:  0.9999048497946893
Test northing MAE:  428.32749050108947
Test easting MAE:  589.3547727957032
[0,2)& 0.48995698185828507 159
[2,3)& 0.7643220137871447 15024
[3,4)& 1.1446933735386653 5535
[4,5)& 0.6680958870221847 228
[5,6)& 1.2686106052514476 96
[6,7)& 1.3260672308544186 48
[7,8)& 1.9000152592779636 42
[8,9)& 1.2252742708609965 59
[9,10)& 1.4673013106759256 45
[10,11)& 1.5206092284980062 42
[11,12)& 1.3143624914764187 39
[12,13)& 1.302299611740889 35
[13,14)& 1.18605793192332 27
[14,15)& 1.42419456479779 41
[15,16)& 1.5693550361802517 18
[16,17)& 2.4213437161465006 24
[17,18)& 2.5370563835107167 16
[18,19)& 1.0976679183973805 9
[19,20)& 1.124847508942318 11
[20,21)& 2.614140510994412 9
[21,22)& 3.4768433740605036 12
[22,23)& 4.315300015296034 11
[23,24)& 1.929164039316909 14
[24,25)& 1.66282514390675

In [ ]:


# Predicting and plotting the result
trainPredict = model.predict(df_trx, batch_size=batch_size)
testPredict = model.predict(df_testx, batch_size=batch_size)

# invert predictions
#(25925, 1) (21585, 1)
trainPredict = np.reshape(trainPredict, (-1, 2))
ytr2d = np.reshape(df_try, (-1, 2))

testPredict = np.reshape(testPredict, (-1, 2))
yte2d = np.reshape(df_testy, (-1, 2))
trainPredict


trainPredict = scaler2.inverse_transform(trainPredict)
trainY = scaler2.inverse_transform(ytr2d)

testPredict = scaler6.inverse_transform(testPredict)
testY = scaler6.inverse_transform(yte2d)

dfpredict = pd.DataFrame(testPredict,columns=['dlon','dlat'])
dftest= pd.DataFrame(testY,columns=['dlon','dlat'])
trainPred= pd.DataFrame(trainPredict,columns=['dlon','dlat'])
trainReal= pd.DataFrame(trainY,columns=['dlon','dlat'])



dfpredictdlon= pd.DataFrame([])
dftestdlon= pd.DataFrame([])
trainPreddlon =pd.DataFrame([])
trainRealdlon = pd.DataFrame([])
dfpredictdlat= pd.DataFrame([])
dftestdlat= pd.DataFrame([])
trainPreddlat = pd.DataFrame([])
trainRealdlat =pd.DataFrame([])

dfpredictdlon['dlon']=dfpredict['dlon']
dftestdlon['dlon']= dftest['dlon']
trainPreddlon['dlon'] = trainPred['dlon']
trainRealdlon['dlon'] = trainReal['dlon']
dfpredictdlat['dlat']= dfpredict['dlat']
dftestdlat['dlat']= dftest['dlat']
trainPreddlat['dlat'] = trainPred['dlat']
trainRealdlat['dlat'] = trainReal['dlat']


dfpredictdlon['lon']=dftestzlon["lon(t-1)"].values
dfpredictdlat['lat'] =dftestzlat["lat(t-1)"].values
dftestdlon['lon']=dftestzlon["lon(t-1)"].values
dftestdlat['lat'] =dftestzlat["lat(t-1)"].values

trainPreddlon['lon'] =dftrainzlon["lon(t-1)"].values
trainRealdlon['lon'] =dftrainzlon["lon(t-1)"].values
trainPreddlat['lat'] =dftrainzlat["lat(t-1)"].values
trainRealdlat['lat'] =dftrainzlat["lat(t-1)"].values



#df.sum(axis = 1, skipna = True)

trainPredlon= pd.DataFrame([])
trainPredlat= pd.DataFrame([])
trainReallon= pd.DataFrame([])
trainReallat= pd.DataFrame([])
dfpredictlon= pd.DataFrame([])
dfpredictlat= pd.DataFrame([])
dftestlon= pd.DataFrame([])
dftestlat= pd.DataFrame([])
dfpredictdlon['flon']=dfpredictdlon.sum(axis = 1, skipna = True)
dfpredictdlat['flat']= dfpredictdlat.sum(axis = 1, skipna = True)
dftestdlon['flon']= dftestdlon.sum(axis = 1, skipna = True)
dftestdlat['flat']= dftestdlat.sum(axis = 1, skipna = True)

trainPreddlon['flon']= trainPreddlon.sum(axis = 1, skipna = True)
trainPreddlat['flat']= trainPreddlat.sum(axis = 1, skipna = True)
trainRealdlon['flon']= trainRealdlon.sum(axis = 1, skipna = True)
trainRealdlat['flat']= trainRealdlat.sum(axis = 1, skipna = True)

trainPredlon['lon']= trainPreddlon['flon']
trainPredlat['lat']= trainPreddlat['flat']
trainReallon['lon']= trainRealdlon['flon']
trainReallat['lat']= trainRealdlat['flat']
dfpredictlon['lon']=dfpredictdlon['flon']
dfpredictlat['lat']=dfpredictdlat['flat']
dftestlon['lon']=dftestdlon['flon']
dftestlat['lat']=dftestdlat['flat']

print("Test easting MSE: ", mean_squared_error(dftestlon, dfpredictlon))

print("Test easting R2: ", r2_score(dftestlon, dfpredictlon))

print("Test northing MSE: ", mean_squared_error(dftestlat, dfpredictlat))

print("Test northing R2: ", r2_score(dftestlat, dfpredictlat))
print("Test easting MSE: ", mean_absolute_error(dftestlon, dfpredictlon))
print("Test northing MSE: ", mean_absolute_error(dftestlat, dfpredictlat))

df_predtrainresults= pd.DataFrame([])
df_trainresults = pd.DataFrame([])
df1= pd.DataFrame([])

df_predtrainresults["long_latPREDICT"] = list(zip(trainPredlon['lon'], trainPredlat['lat']))
df_trainresults["long_latRESULT"] = list(zip(trainReallon['lon'], trainReallat['lat']))

df1['prediction']=df_predtrainresults["long_latPREDICT"]
df1['reality']=df_trainresults["long_latRESULT"]

def distance(df1):
  return math.dist(df1['prediction'],df1['reality'])

df1["distance(m)"] = df1.apply(distance, axis =1)
df1["distance(m)"].mean()
#18886.096412101044 meso sfalma sto train set 18,88609 km  me ola ta matzounia kai 5 epochs

df_predresults = pd.DataFrame([])
df_testresults = pd.DataFrame([])
df= pd.DataFrame([])
df_predresults["long_latPREDICT"] = list(zip(dfpredictlon['lon'], dfpredictlat['lat']))
df_testresults["long_latRESULT"] = list(zip(dftestlon['lon'], dftestlat['lat']))
#!pip install haversine
#from haversine import haversine, Unit

#arr_pred = df_predresults.to_numpy()
#arr_test = df_testresults.to_numpy()

df['prediction']=df_predresults["long_latPREDICT"]
df['reality']=df_testresults["long_latRESULT"]


def distance(df):
  return math.dist(df['prediction'],df['reality'])

df["distance(m)"] = df.apply(distance, axis =1)
df["distance(m)"].mean()


testxdf = scaler5.inverse_transform(df_testx)


dfftestx = pd.DataFrame(testxdf,columns=columns)

df["dt"]=dfftestx['dt(t)']
df
max(df['distance(m)'])


index1=0
index2=0
index3=0
index4=0
index5=0
index6=0
index7=0
index8=0
index9=0
index10=0
index11=0
index12=0
index13=0
index14=0
index15=0
index16=0
index17=0
index18=0
index19=0
index20=0
index21=0
index22=0
index23=0
index24=0
index25=0
index26=0
index27=0
index28=0
index29=0
dist1740=0
dist1680=0
dist1620=0
dist1560=0
dist1500=0
dist1440=0
dist1380=0
dist1320=0
dist1260=0
dist1200=0
dist1140=0
dist1080=0
dist1020=0
dist960=0
dist900=0
dist840=0
dist780=0
dist720=0
dist660=0
dist600=0
dist540=0
dist480=0
dist420=0
dist360=0
dist300=0
dist240=0
dist180=0
dist120=0
dist60=0
for index, row in df.iterrows(): 
  window= row[3]
  if window>=1740 and window<=1800:
          dist1740=dist1740+row[2] 
          index1=index1+1
  if window>=1680 and window<1740:
          dist1680=dist1680+row[2] 
          index2=index2+1
  if window>=1620 and window<1680:
          dist1620=dist1620+row[2] 
          index3=index3+1
  if window>=1560 and window<1620:
          dist1560=dist1560+row[2] 
          index4=index4+1
  if window>=1500 and window<1560:
          dist1500=dist1500+row[2] 
          index5=index5+1
  if window>=1440 and window<1500:
          dist1440=dist1440+row[2] 
          index6=index6+1
  if window>=1380 and window<1440:
          dist1380=dist1380+row[2] 
          index7=index7+1
  if window>=1320 and window<1380:
          dist1320=dist1320+row[2] 
          index8=index8+1
  if window>=1260 and window<1320:
          dist1260=dist1260+row[2] 
          index9=index9+1    
  if window>=1200 and window<1260:
          dist1200=dist1200+row[2] 
          index10=index10+1
  if window>=1140 and window<1200:
          dist1140=dist1140+row[2] 
          index11=index11+1
  if window>=1080 and window<1140:
          dist1080=dist1080+row[2] 
          index12=index12+1
  if window>=1020 and window<1080:
          dist1020=dist1020+row[2] 
          index13=index13+1
  if window>=960 and window<1020:
          dist960=dist960+row[2] 
          index14=index14+1
  if window>=900 and window<960:
          dist900=dist900+row[2] 
          index15=index15+1
  if window>=840 and window<900:
          dist840=dist840+row[2] 
          index16=index16+1
  if window>=780 and window<840:
          dist780=dist780+row[2] 
          index17=index17+1
  if window>=720 and window<780:
          dist720=dist720+row[2] 
          index18=index18+1
  if window>=660 and window<720:
          dist660=dist660+row[2] 
          index19=index19+1
  if window>=600 and window<660:
          dist600=dist600+row[2] 
          index20=index20+1
  if window>=540 and window<600:
          dist540=dist540+row[2] 
          index21=index21+1
  if window>=480 and window<540:
          dist480=dist480+row[2] 
          index22=index22+1
  if window>=420 and window<480:
          dist420=dist420+row[2] 
          index23=index23+1
  if window>=360 and window<420:
          dist360=dist360+row[2] 
          index24=index24+1
  if window>=300 and window<360:
          dist300=dist300+row[2] 
          index25=index25+1
  if window>=240 and window<300:
          dist240=dist240+row[2] 
          index26=index26+1
  if window>=180 and window<240:
          dist180=dist180+row[2] 
          index27=index27+1
  if window>=120 and window<180:
          dist120=dist120+row[2] 
          index28=index28+1
  if window>=0 and window<120:
          dist60=dist60+row[2] 
          index29=index29+1
        
#dist1740,dist1680,dist1620,dist1560,dist1500,dist1440,dist1380,dist1320,dist1200,dist1200,dist1140,
#dist1080,dist1020,dist960,dist900,dist840,dist780,dist720,dist660,dist600,dist540,dist480,dist420,
#dist360,dist300,dist240,dist180,dist120,dist60


print( '[0,2)&' ,(dist60/index29)/1000, index29)
print( '[2,3)&' ,(dist120/index28)/1000,index28)
print( '[3,4)&' ,(dist180/index27)/1000,index27)  
print( '[4,5)&' ,(dist240/index26)/1000,index26)  
print( '[5,6)&' ,(dist300/index25) /1000,index25) 
print( '[6,7)&' ,(dist360/index24) /1000,index24) 
print( '[7,8)&' ,(dist420/index23)/1000,index23)
print( '[8,9)&' ,(dist480/index22) /1000,index22) 
print( '[9,10)&' ,(dist540/index21) /1000,index21) 
print( '[10,11)&' ,(dist600/index20)/1000,index20) 
print( '[11,12)&' ,(dist660/index19) /1000,index19) 
print( '[12,13)&' ,(dist720/index18)/1000,index18)
print( '[13,14)&' ,(dist780 /index17) /1000,index17) 
print( '[14,15)&' ,(dist840/index16) /1000,index16) 
print( '[15,16)&' ,(dist900/index15)/1000,index15) 
print( '[16,17)&' ,(dist960/index14) /1000,index14) 
print( '[17,18)&' ,(dist1020/index13)/1000,index13)
print( '[18,19)&' ,(dist1080/index12) /1000,index12) 
print( '[19,20)&' ,(dist1140/index11) /1000,index11) 
print( '[20,21)&' ,(dist1200/index10)/1000,index10) 
print( '[21,22)&' ,(dist1260/index9)  /1000,index9)
print( '[22,23)&' ,(dist1320 /index8)/1000,index8)
print( '[23,24)&' ,(dist1380/index7) /1000,index7) 
print( '[24,25)&' ,(dist1440/index6)/1000,index6)  
print( '[25,26)&' ,(dist1500/index5)/1000,index5) 
print( '[26,27)&' ,(dist1560/index4)/1000,index4)  
print( '[27,28)&' ,(dist1620/index3)/1000,index3)
print( '[28,29)&' ,(dist1680/index2)/1000,index2)  
print( '[29,30)&' ,(dist1740/index1)/1000,index1)  




675/675 [==============================] - 2s 3ms/step
Test easting MSE:  1060.096619373108
Test easting R2:  0.9992991402253263
Test northing MSE:  668.4945416566013
Test northing R2:  0.999841438079627
distance km under 2 minutes 0.8892305118990407 159
distance km under 3 minutes 1.1982653343434746 15024
distance km under 4 minutes 1.7552382668024975 5535
distance km under 5 minutes 1.594483856349024 228
distance km under 6 minutes 2.3637850271942136 96
distance km under 7 minutes 2.759145039399909 48
distance km under 8 minutes 3.1765236716659353 42
distance km under 9 minutes 2.1281231980439745 59
distance km under 10 minutes 2.5263352612130587 45
distance km under 11 minutes 2.1236552560191115 42
distance km under 12 minutes 1.49325395575635 39
distance km under 13 minutes 2.427047074088517 35
distance km under 14 minutes 1.5423890563890779 27
distance km under 15 minutes 2.3074973388436906 41
distance km under 16 minutes 2.607774204540595 18
distance km under 17 minutes 2.8404315

In [ ]:
import pickle 
with open('32sigmoid16sigmoid8sigmoid4sigmoid2sigmoiddlondlatmodel.pickle','wb') as file:
  pickle.dump(model, file)